# Processing Invoices Using Box's MCP Server

In [23]:
import os
import json
import sqlite3
from dotenv import load_dotenv
from google import genai
from google.genai import types
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client

load_dotenv(override=True)

True

Let's now configure a few settings that we'll use throughout the notebook.

In [ ]:
MODEL_NAME = "gemini-2.5-flash"
BOX_MCP_SERVER_PATH = "/Users/svpino/dev/mcp-server-box"
BOX_MCP_TOOLS = [
    "box_list_folder_content_by_folder_id",
    "box_ai_extract_tool",
]

GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
BOX_FOLDER_ID = os.getenv("BOX_FOLDER_ID")

## Configuring Box' MCP Server

First, let's configure Gemini and prepare the connection to the Box's MCP Server.

In [25]:
client = genai.Client(api_key=GEMINI_API_KEY)

server_params = StdioServerParameters(
    command="uv",
    args=[
        "--directory",
        BOX_MCP_SERVER_PATH,
        "run",
        "src/mcp_server_box.py",
    ],
)

Now, let's create a function to set up the list of MCP tools that will be used to interact with Box.

In [26]:
async def get_mcp_tools(session: ClientSession):
    """
    Set up the list of MCP tools that will be used to interact with Box.
    """
    mcp_tools = await session.list_tools()
    return [
        types.Tool(
            function_declarations=[
                {
                    "name": tool.name,
                    "description": tool.description,
                    "parameters": {p: v for p, v in tool.inputSchema.items()},
                }
            ]
        )
        for tool in mcp_tools.tools
        if tool.name in BOX_MCP_TOOLS
    ]

## Processing invoices

Let's start by defining a helper function that will help us process JSON results from Gemini.

In [27]:
def parse_json(content):
    # Try parsing as pure JSON first
    try:
        return json.loads(content)
    except json.JSONDecodeError:
        pass

    # If that fails, try to extract JSON from the text
    # Look for JSON-like content between curly braces
    import re

    json_match = re.search(r"\{.*\}", content, re.DOTALL)
    if json_match:
        try:
            return json.loads(json_match.group())
        except json.JSONDecodeError:
            pass

    # If all else fails, return the original text
    return content

Since we are going to be using Box's MCP Server, let's create a function that will help us call Gemini using the tools provided by the MCP Server.

In [28]:
async def generate(
    prompt: str, client: genai.Client, session: ClientSession = None, tools: list = None
):
    """
    Generate content using the Gemini model using MCP tools if provided.
    """

    def parse_content(content):
        try:
            return parse_json(content.text)
        except json.JSONDecodeError:
            return content.text

    config = (
        types.GenerateContentConfig(temperature=0, tools=tools)
        if session is not None and tools is not None
        else None
    )

    response = client.models.generate_content(
        model=MODEL_NAME,
        contents=prompt,
        config=config,
    )

    if response.candidates[0].content.parts[0].function_call:
        print(
            f'[Calling MCP Tool: "{response.candidates[0].content.parts[0].function_call.name}"]'
        )
        function_call = response.candidates[0].content.parts[0].function_call
        response = await session.call_tool(
            function_call.name, arguments=dict(function_call.args)
        )

        return [parse_content(content) for content in response.content]

    return response.candidates[0].content.parts[0].text

Let's define a function that extracts the data from a given invoice. This function will use Box's MCP Server to do that.

In [29]:
async def extract_invoice_fields(
    invoice: dict, client: genai.Client, session: ClientSession, tools: list
):
    """
    Extract data from the given invoice.
    """
    print(f'Extracting data from invoice "{invoice["name"]}"...')
    prompt = (
        "Extract the following fields from the invoice "
        f"with file_id {invoice['id']}: "
        "client_name, invoice_amount, product_name. "
        "Return the invoice_amount as a float. "
    )
    response = await generate(prompt, client, session, tools)
    result = json.loads(response[0]["answer"])
    result["file"] = invoice["name"]
    return result


Let's set up the database where we'll store the information of every invoice.

In [30]:
connection = sqlite3.connect("invoices.db")
cursor = connection.cursor()
cursor.execute("""
    CREATE TABLE IF NOT EXISTS invoices (
        file TEXT PRIMARY KEY UNIQUE,
        client TEXT,
        amount REAL,
        product TEXT
    )
""")
connection.commit()

Let's create a Gemini session with access to Box's MCP Server, list the invoices stored in Box and extract the data from each of them.

In [31]:
async with stdio_client(server_params) as (read, write):
    async with ClientSession(read, write) as session:
        await session.initialize()
        tools = await get_mcp_tools(session)

        # Get the list of invoices from the Box folder.
        invoices = await generate(
            f"List the content of the folder with id {BOX_FOLDER_ID}",
            client,
            session,
            tools,
        )
        print(f"Found {len(invoices)} invoices")

        cursor = connection.cursor()

        # Process each invoice and extract the required fields.
        for invoice in invoices:
            invoice_data = await extract_invoice_fields(invoice, client, session, tools)
            print(f"Extracted data: {invoice_data}")

            cursor.execute(
                """
                INSERT INTO invoices (file, client, amount, product)
                VALUES (?, ?, ?, ?)
                ON CONFLICT(file) DO UPDATE SET
                    client=excluded.client,
                    amount=excluded.amount,
                    product=excluded.product
                """,
                (
                    invoice_data["file"],
                    invoice_data["client_name"],
                    invoice_data["invoice_amount"],
                    invoice_data["product_name"],
                ),
            )
        
        connection.commit()

[Calling MCP Tool: "box_list_folder_content_by_folder_id"]
Found 4 invoices
Extracting data from invoice "Invoice-ETOX6AZS-0004.pdf"...
[Calling MCP Tool: "box_ai_extract_tool"]
Extracted data: {'client_name': 'Santiago Valdarrama', 'invoice_amount': 15.0, 'product_name': 'myproduct', 'file': 'Invoice-ETOX6AZS-0004.pdf'}
Extracting data from invoice "Invoice-JV3QQ0DZ-0005.pdf"...
[Calling MCP Tool: "box_ai_extract_tool"]
Extracted data: {'client_name': 'Acme Inc', 'invoice_amount': 400.0, 'product_name': 'Machine Learning School', 'file': 'Invoice-JV3QQ0DZ-0005.pdf'}
Extracting data from invoice "Invoice-JV3QQ0DZ-0006.pdf"...
[Calling MCP Tool: "box_ai_extract_tool"]
Extracted data: {'client_name': 'Acme Inc', 'invoice_amount': 400.0, 'product_name': 'Machine Learning School', 'file': 'Invoice-JV3QQ0DZ-0006.pdf'}
Extracting data from invoice "Wordpress.pdf"...
[Calling MCP Tool: "box_ai_extract_tool"]
Extracted data: {'client_name': 'Test Business', 'invoice_amount': 93.5, 'product_nam

## Generating final reports

Finally, we want to generate a couple of reports with the data that we stored in the database.

In [32]:
print("\nInvoice Report")

cursor = connection.cursor()
cursor.execute("SELECT COUNT(*), SUM(amount) FROM invoices")
total_invoices, total_amount = cursor.fetchone()

print(f"* Total invoices: {total_invoices}")
print(f"* Total amount: {total_amount}")

print("\nBreakdown by client:")
cursor.execute("SELECT client, COUNT(*), SUM(amount) FROM invoices GROUP BY client")
for row in cursor.fetchall():
    client, count, amount = row
    print(f"* {client}: {count} invoices (${amount})")

connection.close()


Invoice Report
* Total invoices: 4
* Total amount: 908.5

Breakdown by client:
* Acme Inc: 2 invoices ($800.0)
* Santiago Valdarrama: 1 invoices ($15.0)
* Test Business: 1 invoices ($93.5)
